In [1]:
!pip install deap

In [2]:
import random
import numpy as np

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli

from tensorflow.keras.datasets import fashion_mnist

2024-06-06 06:46:27.709609: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 06:46:27.709675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 06:46:27.711374: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from copy import deepcopy
from collections import deque, OrderedDict

class DAGValidationError(Exception):
    pass


class DAG(object):
    """ Directed acyclic graph implementation. """

    def __init__(self):
        """ Construct a new DAG with no nodes or edges. """
        self.graph = OrderedDict()

    def add_node(self, node_name, conv=None):
        """ Add a node with an optional conv if it does not exist yet, or error out. """
        if node_name in self.graph:
            raise KeyError(f'node {node_name} allready exists')
        
        self.graph[node_name] = {"edges": set(), 
                                 "conv": conv}
        
    def add_conv(self, node_name, conv):
        if node_name not in self.graph:
            raise KeyError(f'node {node_name} not exists')
        
        self.graph[node_name]['conv'] = conv
    
    def add_node_if_not_exists(self, node_name, conv=None):
        try:
            self.add_node(node_name, conv)
        except KeyError:
            pass

    def delete_node(self, node_name):
        """ Deletes this node and all edges referencing it. """
        if node_name not in self.graph:
            raise KeyError(f'node {node_name} does not exist')
        
        self.graph.pop(node_name)

        for _, data in self.graph.items():
            if node_name in data['edges']:
                data['edges'].remove(node_name)

    def delete_node_if_exists(self, node_name):
        try:
            self.delete_node(node_name)
        except KeyError:
            pass

    def ind_node(self, graph=None):
        """ Returns a list of all nodes in the graph with no dependencies. """
        if graph is None: graph = self.graph

        dependent_nodes = [node for data in graph.values() for node in data['edges']]

        return [node for node in graph.keys() if node not in dependent_nodes]

    def validate(self, graph=None):
        """ Returns (Boolean, message) of whether DAG is valid. """
        graph = graph if graph is not None else self.graph

        if len(self.ind_node(graph)) == 0:
            return (False, 'no independent nodes detected')
        
        try:
            self.topology_sort(graph)
        except ValueError:
            return (False, 'failed topological sort')
        
        return (True, 'valid')
    
    def topology_sort(self, graph=None):
        """ Returns a topological ordering of the DAG.
        Raises an error if this is not possible (graph is not valid).
        Using Kahn algorithm """

        if graph is None: graph = self.graph

        in_degree = {u: 0 for u in graph}
        for u in graph:
            for v in graph[u]['edges']:
                in_degree[v] += 1

        queue = deque([u for u in in_degree if in_degree[u] == 0])

        l = []
        while queue:
            u = queue.pop()
            l.append(u)
            for v in graph[u]['edges']:
                in_degree[v] -= 1
                if in_degree[v] == 0:
                    queue.appendleft(v)

        if len(l) == len(graph):
            return l
        
        raise ValueError('graph is not acyclic')

    def size(self):
        return len(self.graph)

    def add_edge(self, ind_node, dep_node):
        """ Add an edge (dependency) between the specified nodes. """
        if ind_node not in self.graph or dep_node not in self.graph:
            raise KeyError('one or more nodes do not exist in graph')
        
        test_graph = deepcopy(self.graph)
        test_graph[ind_node]['edges'].add(dep_node)
        is_valid, message = self.validate(test_graph)

        del test_graph

        if is_valid:
            self.graph[ind_node]['edges'].add(dep_node)
        else:
            raise ValueError(message)
        

    def delete_edge(self, ind_node, dep_node):
        """ Delete an edge from the graph. """
        if dep_node not in self.graph.get(ind_node, {'edges': set()})['edges']:
            raise KeyError('this edge does not exist in graph')
        
        self.graph[ind_node]['edges'].remove(dep_node)

    def rename_edges(self, old_task_name, new_task_name):
        """ Change references to a task in existing edges. """
        for node, data in self.graph.items():
            if node == old_task_name:
                self.graph[new_task_name] = data
                del self.graph[old_task_name]
            else:
                if old_task_name in data['edges']:
                    data['edges'].remove(old_task_name)
                    data['edges'].add(new_task_name)

    def predecessors(self, node):
        """ Returns a list of all predecessors of the given node """
        return [key for key, data in self.graph.items() if node in data['edges']]
    
    def downstream(self, node):
        """ Returns a list of all nodes this node has edges towards. """
        if node not in self.graph:
            raise KeyError(f'node {node} is not in graph')
        
        return list(self.graph[node]['edges'])
    
    def all_downstreams(self, node):
        """ Returns a list of all nodes ultimately downstream
        of the given node in the dependency graph, in
        topological order. """
        nodes = [node]
        nodes_seen = set()
        i = 0

        while i < len(nodes):
            downstreams = self.downstream(node[i])
            for downstream_node in downstreams:
                if downstream_node not in nodes_seen:
                    nodes_seen.add(downstream_node)
                    nodes.append(downstream_node)

            i += 1

        return list(filter(lambda node: node in nodes_seen, self.topology_sort()))
    
    def all_leaves(self):
        """ Return a list of all leaves (nodes with no downstreams) """
        return [key for key, data in self.graph.items() if not data['edges']]
    
    def from_dict(self, graph_dict):
        """ Reset the graph and build it from the passed dictionary.
        The dictionary takes the form of {node_name: [directed edges]} """
        self.graph = OrderedDict()

        for new_node in graph_dict.keys():
            self.add_node(new_node)

        for ind_node, dep_nodes in graph_dict.items():
            if not isinstance(dep_node, list):
                raise TypeError('dict values must be lists')
            for dep_node in dep_nodes:
                self.add_edge(ind_node, dep_node)

    def get_conv(self, node_name):
        """ Return the conv associated with a node """
        if node_name in self.graph:
            return self.graph[node_name]['conv']
        else:
            raise KeyError(f'node {node_name} does not exist')


# dag = DAG()
# dag.add_node('node1', tf.constant([1, 2, 3]))
# dag.add_node('node2', tf.constant([4, 5, 6]))
# dag.add_edge('node1', 'node2')

# print(dag.get_conv('node1'))  # Output: tf.conv([1 2 3], shape=(3,), dtype=int32)
# print(dag.get_conv('node2'))  # Output: tf.conv([4 5 6], shape=(3,), dtype=int32)

In [4]:
import numpy as np
import tensorflow as tf


def generate_dag(optimal_indvidual, stage_name, num_nodes):
    nodes = np.empty((0), dtype = str)

    for n in range(1, num_nodes + 1):
        nodes = np.append(nodes, f'{stage_name}_{str(n)}')

    dag = DAG()
    for n in nodes:
        dag.add_node(n)

    edges = np.split(optimal_indvidual, np.cumsum(range(num_nodes - 1)))[1:]

    v2 = 2
    for e in edges:
        v1 = 1
        for i in e:
            if i:
                dag.add_edge(f'{stage_name}_{str(v1)}', f'{stage_name}_{str(v2)}')
            v1 += 1
        v2 += 1

    for n in nodes:
        if len(dag.predecessors(n)) == 0 and len(dag.downstream(n)) == 0:
            dag.delete_node(n)
            nodes = np.delete(nodes, np.where(nodes == n)[0][0])

    return dag, nodes

def has_same_elements(x):
    return len(set(x)) <= 1

def add_convolution(input_tensor, layer_name, kernel_height=5, kernel_width=5, num_channels=1, depth=1):
    conv_layer = tf.keras.layers.Conv2D(
        filters=depth,
        kernel_size=(kernel_height, kernel_width),
        strides=(1, 1),
        padding="same",
        activation='relu',
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1),
        bias_initializer=tf.keras.initializers.Constant(0.01),
        name=layer_name
    )(input_tensor)

    return tf.keras.layers.BatchNormalization(name=f'{layer_name}_bn')(conv_layer)

def apply_pooling(input_tensor, pool_size=(16, 16), stride_size=(2, 2)):
    pooling_layer = tf.keras.layers.MaxPooling2D(pool_size=pool_size, strides=stride_size, padding='same')

    return pooling_layer(input_tensor)


def generate_model(individual, stages, num_nodes, bits_indices, filters, num_labels):
    tf.keras.backend.clear_session()

    input_tensor = tf.keras.Input(shape=(28, 28, 1), name='input_tensor')

    d_node = input_tensor
    for stage_name, num_node, bpi, filter in zip(stages, num_nodes, bits_indices, filters):
        indv = individual[bpi[0]:bpi[1]]
        d_node = add_convolution(d_node, f'{stage_name}_input', depth=filter)

        if not has_same_elements(indv):
            dag, nodes = generate_dag(indv, stage_name, num_node)
            without_predessors = dag.ind_node()
            without_successors = dag.all_leaves()

            input_stage = d_node
            for wop in without_predessors:
                d_node = add_convolution(input_stage, f'{wop}_conv', depth=filter)
                dag.add_conv(wop, d_node)

            for n in nodes:
                predessors = dag.predecessors(n)

                if len(predessors) == 0: continue
                elif len(predessors) > 1:
                    sum_conv = dag.get_conv(predessors[0])
                    for prd in range(1, len(predessors)):
                        sum_conv = tf.keras.layers.Add()([sum_conv, dag.get_conv(predessors[prd])])

                    d_node = add_convolution(sum_conv, f'{n}_conv', depth=filter)
                    dag.add_conv(n, d_node)
                elif predessors:
                    d_node = add_convolution(dag.get_conv(predessors[0]), f'{n}_conv', depth=filter)
                    dag.add_conv(n, d_node)

            if len(without_successors) > 1:
                sum_conv = dag.get_conv(without_successors[0])
                for suc in range(1, len(without_successors)):
                    sum_conv = tf.keras.layers.Add()([sum_conv, dag.get_conv(without_successors[suc])])

                d_node = add_convolution(sum_conv, f'{stage_name}_output', depth=filter)
            else:
                d_node = add_convolution(dag.get_conv(without_successors[0]), f'{stage_name}_output', depth=filter)

        d_node = apply_pooling(d_node)

    d_node = tf.keras.layers.Flatten()(d_node)
    d_node = tf.keras.layers.Dense(64, activation='relu')(d_node)
    output_tensor = tf.keras.layers.Dense(num_labels, activation='softmax')(d_node)

    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    
    return model

In [5]:
def get_bit_idx(num_nodes):
    L =  0 # genome length
    BITS_INDICES = np.empty((0, 2),dtype = np.int32)
    start = 0
    end = 0

    for x in num_nodes:
        end = end + sum(range(x))
        BITS_INDICES = np.vstack([BITS_INDICES, [start, end]])
        start = end
    L = end

    return L, BITS_INDICES

In [6]:
# Load the dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the images to a range of 0 to 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape the data to include the channel dimension
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))


In [7]:
STAGES = np.array(["s1", "s2"]) # S
NUM_NODES = np.array([4, 5]) # K
NUM_LABELS = 10
FILTERS = np.array([32, 64, 128])
L, BITS_INDICES = get_bit_idx(NUM_NODES)

POPULATION_SIZE = 20
NUM_GEN = 5

TRAINING_EPOCHS = 8

In [8]:
def evaluate_model(individual):
    model = generate_model(individual, STAGES, NUM_NODES, BITS_INDICES, FILTERS, NUM_LABELS)

    model.fit(train_images, train_labels, epochs=TRAINING_EPOCHS, validation_data=(test_images, test_labels))
    
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    
    return test_acc, 


In [ ]:
random.seed(11)

# Kiểm tra creator đã được khởi tạo với hai lớp FitnessMax và Individual hay chưa
if not hasattr(creator, "FitnessMax"):
    # Tạo ra một lớp FitnessMax mở rộng từ lớp Fitness (base class của DEAP),
    # được sử dụng để tối đa hoá đơn mục tiêu, giá trị Fitness càng cao càng tốt (weights=(1.0,))
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if not hasattr(creator, "Individual"):
    # Tạo ra một lớp Individual có dạng là list, thuộc tính fitness là một instance của FitnessMax
    creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()
hof = tools.HallOfFame(POPULATION_SIZE * L)

# Đăng ký một hàm có tên là binary được dùng để tạo ra một giá trị
#nhị phân theo phân phối bernoulli với xác suất 0.5 cho 1 và 0.5 cho 0
toolbox.register("binary", bernoulli.rvs, 0.5) 

# Đăng lý một hàm có tên là individual được gọi lặp lại để tạo ra một cá thể có L biến nhị phân,
# sử dụng lớp Individual được tạo ở trên 
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binary, n = L)

# Đăng ký một hàm có tên là population được gọi lặp lại để tạo ra một quần thể các cá thể (list individual), 
# mỗi cá thể được tạo ra bằng hàm individual
toolbox.register("population", tools.initRepeat, list , toolbox.individual)

# Đăng ký một hàm lai ghép có tên là mate sử dụng hàm cxOrdered để hoán vị các biến
toolbox.register("mate", tools.cxOrdered)

# Đăng ký một hàm đột biến tên là mutate, sử dụng hàm mutShuffleIndexes để hoán đổi các index của các biến
# với xác suất đột biến là 0.8
toolbox.register("mutate", tools.mutShuffleIndexes, indpb = 0.8)

# Đăng ký một hàm chọn lọc với tên là select, sử dụng chiến lược chọn lọc roulette wheel
toolbox.register("select", tools.selRoulette)

# Đăng ký một hàm đánh giá tên là evaluate sử dụng hàm evaluateModel được định nghĩa ở trên
toolbox.register("evaluate", evaluate_model)

# Tạo ra một population với số lượng các cá thể là n
popl = toolbox.population(n = POPULATION_SIZE)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Sử dụng simple evolutionary algorithm được cung cấp với DEAP
# với popl được tạo ra ở trên
# xác suất lai ghép là 0.4
# xác suất đột biến là 0.05
# số lượng thế hệ
# và in ra thông tin của quá trình thực thi

pop, log = algorithms.eaSimple(popl, toolbox, cxpb = 0.4, mutpb = 0.05, ngen = NUM_GEN, stats=stats, halloffame=hof, verbose = True)

Epoch 1/8
  10/1875 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - accuracy: 0.1614 - loss: 13.6139 

I0000 00:00:1717656406.838500     873 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717656406.865807     873 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1873/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7077 - loss: 1.1707

W0000 00:00:1717656429.026968     873 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 38s 13ms/step - accuracy: 0.7078 - loss: 1.1699 - val_accuracy: 0.7865 - val_loss: 0.5594
Epoch 2/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.8522 - loss: 0.4002 - val_accuracy: 0.8173 - val_loss: 0.5523
Epoch 3/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.8856 - loss: 0.3160 - val_accuracy: 0.8570 - val_loss: 0.3702
Epoch 4/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.8951 - loss: 0.2947 - val_accuracy: 0.8691 - val_loss: 0.3903
Epoch 5/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9038 - loss: 0.2706 - val_accuracy: 0.8850 - val_loss: 0.3237
Epoch 6/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9103 - loss: 0.2506 - val_accuracy: 0.8838 - val_loss: 0.3240
Epoch 7/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9144 - loss: 0.2399 - val_accuracy: 0.9051 - val_loss: 0.2652
Epoch 8/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9229 - loss: 0.2163 - va

...

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 14ms/step - accuracy: 0.6737 - loss: 1.0597 - val_accuracy: 0.7504 - val_loss: 0.8943
Epoch 2/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - accuracy: 0.8640 - loss: 0.3749 - val_accuracy: 0.8680 - val_loss: 0.3825
Epoch 3/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.8836 - loss: 0.3150 - val_accuracy: 0.8756 - val_loss: 0.3413
Epoch 4/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 13ms/step - accuracy: 0.8985 - loss: 0.2790 - val_accuracy: 0.8788 - val_loss: 0.3487
Epoch 5/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.9065 - loss: 0.2576 - val_accuracy: 0.8965 - val_loss: 0.3014
Epoch 6/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.9156 - loss: 0.2344 - val_accuracy: 0.8987 - val_loss: 0.2787
Epoch 7/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.9189 - loss: 0.2246 - val_accuracy: 0.8957 - val_loss: 0.3061
Epoch 8/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - accuracy: 0.9264 - loss: 0.2084 - va

W0000 00:00:1717667264.989470     873 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1872/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7227 - loss: 0.9872

W0000 00:00:1717667289.878457     873 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.7229 - loss: 0.9863 - val_accuracy: 0.8310 - val_loss: 0.4697
Epoch 2/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.8715 - loss: 0.3561 - val_accuracy: 0.7945 - val_loss: 0.6381
Epoch 3/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.8904 - loss: 0.3045 - val_accuracy: 0.8673 - val_loss: 0.3693
Epoch 4/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.9029 - loss: 0.2691 - val_accuracy: 0.8694 - val_loss: 0.3574
Epoch 5/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.9112 - loss: 0.2471 - val_accuracy: 0.9016 - val_loss: 0.2761
Epoch 6/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.9181 - loss: 0.2337 - val_accuracy: 0.9078 - val_loss: 0.2588
Epoch 7/8
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.9200 - loss: 0.2207 - val_accuracy: 0.9085 - val_loss: 0.2583
Epoch 8/8
 431/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - accuracy: 0.9311 - loss: 0.1930

In [ ]:
import matplotlib.pyplot as plt
generations = np.array([entry['gen'] for entry in log])
avg_values = np.array([entry['avg'] for entry in log])
std_values = np.array([entry['std'] for entry in log])
min_values = np.array([entry['min'] for entry in log])
max_values = np.array([entry['max'] for entry in log])

# Vẽ biểu đồ
plt.figure(figsize=(8, 6))

# Biểu đồ với dữ liệu đã log
plt.errorbar(generations, avg_values, yerr=[avg_values - min_values, max_values - avg_values], fmt='o-', label='Data Log', color='blue')

# Thiết lập nhãn và tiêu đề
plt.xlabel('Generation Number')
plt.ylabel('Accuracy')
plt.title(f'Accuracy over Generations with pop_size={POPULATION_SIZE},generations={NUM_GEN},epochs={TRAINING_EPOCHS}')
plt.legend()
plt.grid(True)
plt.savefig('Plot.png', format='png')
# Hiển thị biểu đồ
plt.show()

In [ ]:
best_individuals = tools.selBest(popl, k = 1)
for bi in best_individuals:
    print(bi, " ", bi.fitness.values[0])

In [ ]:
model = generate_model(best_individuals[0], STAGES, NUM_NODES, BITS_INDICES, FILTERS, NUM_LABELS)
model.save(f'{POPULATION_SIZE}_{NUM_GEN}_{TRAINING_EPOCHS}.h5')
model.summary()

In [ ]:
# # print top optimal solutions 
# best_individuals = tools.selBest(popl, k = 1)
# for bi in best_individuals:
#     print(bi.fitness.values[0])